In [1]:
import folium
import pandas as pd
import numpy as np
import tkinter as tk
import webbrowser
import random
from sklearn.cluster import KMeans
from PyQt5.QtCore import QUrl


In [2]:
# 读取数据
data = pd.read_csv('loc_information.csv', encoding="GBK")

In [3]:
# 计算玩几个景点
def cal_nloc(time_day,trans):
    if time_day == '0.5':
        num = 2
    elif time_day == '1':
        num = 3
    elif time_day == '2':
        num = 6
    elif time_day == '3' :
        num = 9
    elif time_day == '4' :
        num = 12
    else:
        num = 15

    if trans == "步行":
        bonus = 1.0
    elif trans == "单车" :
        bonus = 1.1
    elif trans == "巴士" or trans == "均可" or trans == "请选择" :
        bonus = 1.3
    else:
        bonus = 1.5
    # 加上起点和终点
    nloc = bonus * num + 2
    return int(nloc)

In [4]:
def cal_score(selected_clusters, trans, money, focus, nloc):
    # 初始化每个地点的分数
    location_scores = {index: 0 for index, _ in data.iterrows()}
 
    # 对钱数进行处理
    if money == "0":
        max_money = 0
    elif money =="<50":
        max_money = 50
    elif money =="50-100":
        max_money = 100
    elif money =="100-200":
        max_money = 200
    elif money =="200-500":
        max_money = 500
    elif money =="500+":
        max_money = 10000
        

    # 计算每个地点的分数
    for cluster in selected_clusters:
        cluster_data = data[data['cluster'] == cluster]
        # 聚类内部的地点加分
        for index, row in cluster_data.iterrows():
            location_scores[index] += 50000/nloc**2  # 聚类里的地点加分
            
    for index, row in data.iterrows():

        # 高价格惩罚
        price_not_null=row['price'] if pd.notnull(row['price']) else 0
        if max_money < price_not_null:
            location_scores[index] -= 10000
        elif max_money//nloc < price_not_null:
            location_scores[index] -= (price_not_null-max_money//nloc) * 2
            
            
        # 设置评价指标
        # 价格参数
        price = (price_not_null + 1) ** 0.5
        # 景区热度
        hotscore = (row['counts'] + 0.2* row['comments'] + 0.5 * row['reposts'])//100
        # 交通拥堵
        if trans == '单车' or trans == '步行':
            congest_effect = 0.1
        else:
            congest_effect = 1
            
        congest = pd.read_csv('congestion.csv', encoding='GBK')
        
        # 确保正确匹配地点信息，并且处理可能的缺失值
        place_congest = congest[congest['annotation_place_title'] == row['location']]
        if not place_congest.empty:
            congestion = place_congest['counts'].iloc[0] * 2 * congest_effect
        else:
            # 处理无法匹配到地点信息的情况，可以根据具体需求设置默认值或者其他处理方式
            congestion = 0  # 默认设置为0，表示无拥堵

    
        # 评分指标(待补全)
        attitudes = (row['attitudes']+1)//3

        # 根据focus设置不同公式
        if focus == "性价比":
            location_scores[index] += attitudes//price * 3 + hotscore - congestion 
        elif focus == "避人流":
            location_scores[index] += attitudes//price - 0.5 * hotscore - 1.5 * congestion
        elif focus == "网红打卡":
            location_scores[index] += attitudes//price + 3 * hotscore - 0.5 * congestion
        elif focus == "游玩体验":
            location_scores[index] += attitudes*1.5 - price * 0.5 + hotscore - 1.5 * congestion
        else:
            location_scores[index] += attitudes//price  + hotscore - congestion      
    return location_scores      
    


In [5]:
def generate_map_based_on_preferences(option_vars, start_loc, end_loc):
    # 创建一个空的列表来存储符合条件的景点

    num_clusters = 6
    filtered_locations = []
    time_day = option_vars['游玩时间：']
    trans = option_vars['偏好交通方式：']
    money = option_vars["景点预算："]
    focus = option_vars['游玩最关注什么：']
    interest_types = option_vars['感兴趣类型：']

    nloc = cal_nloc(time_day,trans)
    
    # 聚类景点
    locations = np.array(data[['lat', 'lon']])
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(locations)
    data['cluster'] = kmeans.labels_
    
    # 根据时间天数确定选择的聚类数量
    if time_day == '0.5' or time_day == '1':
        num_selected_clusters = 1
    elif time_day == '2':
        num_selected_clusters = 2
    elif time_day == '3' or time_day == '4':
        num_selected_clusters = 3
    else:
        num_selected_clusters = num_clusters
    
    # 计算每个聚类中所有元素到起始点的平均距离之和
    avg_distances_to_start = {}
    avg_distances_to_end = {}
    for cluster in range(num_clusters):
        cluster_data = data[data['cluster'] == cluster]
        # 计算到起始点的平均距离
        start_loc_lat_lon = np.array([start_loc['lat'], start_loc['lon']])
        distances_to_start = np.linalg.norm(cluster_data[['lat', 'lon']].values - start_loc_lat_lon, axis=1)
        avg_distances_to_start[cluster] = np.mean(distances_to_start)
        
        # 计算到结束点的平均距离（如果有）
        end_loc_lat_lon = np.array([end_loc['lat'], end_loc['lon']])
        distances_to_end = np.linalg.norm(cluster_data[['lat', 'lon']].values - end_loc_lat_lon, axis=1)
        avg_distances_to_end[cluster] = np.mean(distances_to_end)
            
    # 对平均距离进行排序
    sorted_clusters = sorted(range(num_clusters), key=lambda x: (avg_distances_to_start[x] + avg_distances_to_end[x], avg_distances_to_start[x], avg_distances_to_end[x]))
    
    # 根据时间天数选择最小的聚类
    selected_clusters = sorted_clusters[:num_selected_clusters]
    
    # 计算每个地点的分数
    location_scores = cal_score(selected_clusters, trans, money, focus, nloc)
    
    # 根据分数对地点进行排序
    sorted_locations_by_scores = sorted(location_scores.items(), key=lambda x: x[1], reverse=True)

    
    # 选择得分最高的前 nloc个地点
    for index, score in sorted_locations_by_scores[:nloc]:
        filtered_locations.append(data.loc[index].to_dict())
    
    # 输出筛选后的前 nloc 个景点
    return filtered_locations


In [6]:
class GeneticAlgorithm:
    def __init__(self, locations, kmeans, cluster_color):
        self.locations = locations
        self.num_locations = len(locations)
        self.population_size = 300
        self.mutation_rate = 0.02
        self.num_generations = 6000
        self.kmeans = kmeans
        self.cluster_color = cluster_color

    def generate_initial_population(self):
        population = []
        for _ in range(self.population_size):
            path = random.sample(self.locations, self.num_locations)
            population.append(path)
        return population

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(0, self.num_locations - 1)
        child1 = parent1[:crossover_point] + [loc for loc in parent2 if loc not in parent1[:crossover_point]]
        child2 = parent2[:crossover_point] + [loc for loc in parent1 if loc not in parent2[:crossover_point]]
        return child1, child2

    def mutate(self, path):
        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(self.num_locations), 2)
            path[idx1], path[idx2] = path[idx2], path[idx1]
        return path

    def calculate_path_length(self, path, start_location, end_location):
        total_distance = 0
        path_with_start_end = [start_location] + path + [end_location]
        for i in range(len(path_with_start_end) - 1):
            total_distance += self.get_distance(path_with_start_end[i], path_with_start_end[i+1])
        return total_distance

    def get_distance(self, loc1, loc2):
        lat1, lon1 = loc1['lat'], loc1['lon']
        lat2, lon2 = loc2['lat'], loc2['lon']
        distance = ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5
        return distance

    def optimize(self, start_location, end_location):
        population = self.generate_initial_population()

        for _ in range(self.num_generations):
            # 选择父代
            parents = random.sample(population, 2)
            parent1, parent2 = parents[0], parents[1]

            # 杂交
            child1, child2 = self.crossover(parent1, parent2)

            # 变异
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)

            # 用子代替换父代
            population.extend([child1, child2])

            # 选择最优个体
            population = sorted(population, key=lambda x: self.calculate_path_length(x, start_location, end_location))[:self.population_size]

        # 去掉起始点和结束点并返回最优个体
        best_path = population[0]
        return best_path[1:-1]


In [7]:
import itertools

def distance(loc1, loc2):
    lat1, lon1 = loc1['lat'], loc1['lon']
    lat2, lon2 = loc2['lat'], loc2['lon']
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)
    
# 递归函数，将路线分为指定数量的组，并计算每组中路线总长度的方差
def find_min_variance_routes(start_location, end_location, paths, num_groups):
    print(paths)
    if num_groups == 1:
        # 如果只剩下一组，直接计算其总长度的方差
        total_distances = [sum(distance(loc1, loc2) for loc1, loc2 in zip(day[:-1], day[1:])) for day in paths]
        variance = np.var(total_distances)
        return variance, [paths]
    else:
        min_variance = float('inf')
        best_partition = []
        
        # 生成所有可能的分组情况
        for partition in itertools.permutations(range(1, len(paths)), num_groups - 1):
            
            partition = [0] + list(partition) + [len(paths)]
            groups = [paths[partition[i]:partition[i+1]] for i in range(num_groups)]
            if sum([len(group) for group in groups]) == len(paths):
                # 将前一个组的最后一个景点添加到当前组的起始位置
                for i in range(1, len(groups)):
                    groups[i].insert(0, paths[partition[i]-1])
                
                # 添加起点和终点
                groups[0].insert(0, start_location)
                groups[-1].append(end_location)
                
                total_distances = [
                    sum(distance(loc1, loc2) for loc1, loc2 in zip(day[:-1], day[1:])) for day in groups
                ]
                
                total_variance = np.var(total_distances)
                if total_variance < min_variance:
                    min_variance = total_variance
                    best_partition = groups
                    # 输出每组中景点的个数
        for i in range(1, len(best_partition)):
            best_partition[i].pop(0)
        best_partition[0].pop(0)
        best_partition[-1].pop()
        return min_variance, best_partition

In [ ]:
import sys
import math
import pandas as pd
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton, QComboBox, QListWidget, QSizePolicy, QFrame
from PyQt5.QtWebEngineWidgets import QWebEngineView
import webbrowser

import math
def read_start_locations():
    # Read locations from start_loc.csv
    start_loc_data = pd.read_csv('start_loc.csv', encoding="GBK")
    start_locations = list(start_loc_data['location'])
    return start_locations
    
def read_end_locations():
    # Read locations from start_loc.csv
    end_loc_data = pd.read_csv('start_loc.csv', encoding="GBK")
    end_locations = list(end_loc_data['location'])
    
    return end_locations
    
def linear_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def find_restaurants(locations, restaurant_data):
    # 存储每个餐馆到所有位置的总距离
    restaurants = []
    record={}
    for index, row in restaurant_data.iterrows():
        restaurant_name = row['name']  # 餐馆名称
        restaurant_location = (row['lat'], row['lon'])  # 餐馆位置
        # 计算该餐馆到所有地点的总距离
        for place in locations:
            location=(place['lat'],place['lon'])
            distance = linear_distance(location, restaurant_location)
            if distance<=0.05:
                if 'location' not in record:
                    record['location'] = 1
                else:
                    record['location']+=1
                if record['location']<=10:
                    restaurants.append((restaurant_name,restaurant_location))
                    break
    return restaurants

    
def find_nearest_hotels(location, hotel_data,n):
    # 计算位置与每个酒店的距离，并存储为 (酒店名称, 距离) 的元组列表
    distances = []
    for index, row in hotel_data.iterrows():
        hotel_name = row['location']  
        hotel_location = (row['lat'], row['lon'])
        distance = linear_distance(location, hotel_location)  
        distances.append((hotel_name, distance,hotel_location))
    
    # 按距离排序
    sorted_distances = sorted(distances, key=lambda x: x[1])
    
    # 返回最近的五个酒店
    nearest_hotels = sorted_distances[:n]
    
    return nearest_hotels


def generate_map(locations, start_location, end_location, touring_time):
    # 读取地点信息数据
    data = pd.read_csv('loc_information.csv', encoding="GBK")
    # 提取经纬度数据
    coordinates = data[['lat', 'lon']]
    coordinates.columns = ['lat', 'lon']

    # 使用K均值算法进行聚类
    kmeans = KMeans(n_clusters=6, random_state=0).fit(coordinates)

    # 将聚类结果添加到原始数据中
    data['cluster'] = kmeans.labels_

    # 创建颜色字典，将聚类序号映射到颜色
    cluster_color = {cluster: color for cluster, color in zip(range(6), ['red', 'blue', 'green', 'orange', 'purple', 'black'])}

    # 创建地图对象
    m = folium.Map(location=[data['lat'].mean()-0.10, data['lon'].mean()+0.2], zoom_start=10)

    location_introduction = dict(zip(data['location'], data['introduction']))
    
    # 标记起始点
    start_marker = folium.CircleMarker(location=[start_location['lat'], start_location['lon']], radius=12, color='yellow', fill=True, fill_color='yellow', fill_opacity=0.7)
    start_marker.add_to(m)
    
    # 标记结束点
    end_marker = folium.CircleMarker(location=[end_location['lat'], end_location['lon']], radius=12, color='gray', fill=True, fill_color='gray', fill_opacity=0.7)
    end_marker.add_to(m)

    # 使用遗传算法优化路径并计算每天行走距离方差最小的情况下，每天走几个景点
    genetic_algorithm = GeneticAlgorithm(locations, kmeans, cluster_color)
    optimized_path = genetic_algorithm.optimize(start_location,end_location)
    if touring_time != 1: 
        min_variance, best_partition = find_min_variance_routes(start_location, end_location, optimized_path, touring_time)
    else:
        best_partition = [optimized_path]
    best_daily_visits = {f"Day {i+1}": group for i, group in enumerate(best_partition)}

    # 分割最优路径并绘制每天的路线和点
    day_line_colors = {1: 'blue', 2: 'green', 3: 'red', 4: 'orange', 5: 'purple', 6: 'black'}
    start_index = 0
    for day, visit_count in best_daily_visits.items():
        end_index = start_index + len(visit_count)
        day_group = folium.FeatureGroup(name=day)
        
        # 添加酒店选项
        hotel_group = folium.FeatureGroup(name=f"{day} Hotels")
        day_group.add_child(hotel_group)
        restaurant_group = folium.FeatureGroup(name=f"{day} Hotels")
        day_group.add_child(restaurant_group)
    
        # 绘制每天的路线
        if day == "Day 1":
            # 如果是第一天，则连接起始点和第一个点
            folium.PolyLine(locations=[(start_location['lat'], start_location['lon']), (optimized_path[start_index]['lat'], optimized_path[start_index]['lon'])], color=day_line_colors[1], tooltip="Day 1").add_to(day_group)
        
        if day == f"Day {touring_time}":
            folium.PolyLine(locations=[(optimized_path[end_index-1]['lat'], optimized_path[end_index-1]['lon']), (end_location['lat'], end_location['lon'])], color=day_line_colors[touring_time], tooltip=f"Day {touring_time}").add_to(day_group)
    
        for i in range(start_index, end_index - 1):
            loc1 = (optimized_path[i]['lat'], optimized_path[i]['lon'])
            loc2 = (optimized_path[i+1]['lat'], optimized_path[i+1]['lon'])
            folium.PolyLine(locations=[loc1, loc2], color=day_line_colors[int(day.split()[1])], tooltip=day).add_to(day_group)
        
        # 绘制每天的点
        for i in range(start_index, end_index):
            location = optimized_path[i]
            loc = (location['lat'], location['lon'])
            location_name = location['location']
            counts = location['counts']
            
            # 获取地点所在的聚类序号
            cluster = kmeans.predict([[loc[0], loc[1]]])[0]
    
            # 获取地点的颜色
            color = cluster_color[cluster]
    
            # 添加地点标记
            marker = folium.Marker(loc, popup=f"{location_name}: {counts} 次访问", icon=folium.Icon(color=color))
            marker.add_to(day_group)
            
            # 获取地点介绍文本
            introduction_text = location_introduction.get(location_name, "暂无介绍")
    
            # 在弹出窗口中显示地点介绍文本
            html = f"<h3>{location_name}</h3><p>{introduction_text}</p>"
            folium.Popup(html, max_width=400).add_to(marker)
    
        # 连接每天的点
        if start_index != 0:
            loc_prev_day = (optimized_path[start_index - 1]['lat'], optimized_path[start_index - 1]['lon'])
            loc = (optimized_path[start_index]['lat'], optimized_path[start_index]['lon'])
            folium.PolyLine(locations=[loc_prev_day, loc], color=day_line_colors[int(day.split()[1])], tooltip=day).add_to(day_group)
     
        # 将每天的图层添加到地图上
        day_group.add_to(m)
        if day != f"Day {touring_time}":
            # 找到每天的最后一个点
            last_location = optimized_path[end_index - 1]
            # 获取最近的酒店
            hotel_data = pd.read_csv('hotel_max.csv',encoding = 'GBK')
            n_hotel = 5
            nearest_hotels = find_nearest_hotels((last_location['lat'], last_location['lon']), hotel_data, n_hotel)
            # 在地图上添加酒店标记
            for i, hotel in enumerate(nearest_hotels, 1):
                hotel_name, distance, hotel_location = hotel
                distance = linear_distance((last_location['lat'], last_location['lon']), hotel_location)
                folium.RegularPolygonMarker(location=hotel_location, popup=hotel_name,fill_color = 'blue',number_of_sides=3,radius = 15).add_to(hotel_group)

        # 获取最近的饭店
        restaurant_data = pd.read_csv('restaurant.csv',encoding = 'GBK')
        restaurants_choice = find_restaurants(optimized_path[start_index:(end_index - 1)], restaurant_data)
        # 在地图上添加饭店标记
        for i, restaurant in enumerate(restaurants_choice, 1):
            restaurant_name, restaurant_location = restaurant
            folium.RegularPolygonMarker(location=restaurant_location, popup=restaurant_name,color='green',fill_color = 'green',number_of_sides=4,radius = 8).add_to(restaurant_group)


        start_index = end_index

    # 将图层控制添加到地图中
    folium.LayerControl(collapsed=False).add_to(m)

    # 将地图保存为HTML文件
    m.save('C:/Users/33549/my_tasks_all/database_final_work/optimized_map.html')




def confirm_options(option_vars, right_layout, right_widget):
    selected_options = {}
    for label_text, var in option_vars.items():
        if isinstance(var,QComboBox):
            selected_options[label_text] = var.currentText()
        else:
            print('yes')
            selected_options[label_text] = [item.text() for item in var.selectedItems()]
    data = pd.read_csv('start_loc.csv', encoding="GBK")
    start = selected_options.pop("起始点：")
    end = selected_options.pop("结束点：")
    
    loc_data = pd.read_csv('start_loc.csv', encoding="GBK")
    start_loc_filtered = loc_data[loc_data['location'] == start]
    start_location = {
        'location': start_loc_filtered['location'].iloc[0],
        'lat': start_loc_filtered['lat'].iloc[0],
        'lon': start_loc_filtered['lon'].iloc[0]
    }
    
    end_loc_filtered = loc_data[loc_data['location'] == end]
    end_location = {
        'location': end_loc_filtered['location'].iloc[0],
        'lat': end_loc_filtered['lat'].iloc[0],
        'lon': end_loc_filtered['lon'].iloc[0]
    }
    
    time_day = selected_options['游玩时间：']
    filtered_locations = generate_map_based_on_preferences(selected_options,start_location,end_location)
    generate_map(filtered_locations, start_location, end_location, math.ceil(float(time_day)))
    
    # 删除之前右侧布局中的所有部件
    for i in reversed(range(right_layout.count())):
        widget = right_layout.itemAt(i).widget()
        if widget is not None:
            widget.deleteLater()

    # 显示新的地图
    map_view = QWebEngineView()
    map_view.load(QUrl.fromLocalFile("C:/Users/33549/my_tasks_all/database_final_work/optimized_map.html"))
    map_view.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
    right_layout.addWidget(map_view)
    right_widget.setLayout(right_layout)


def create_gui():
    app = QApplication(sys.argv)
    main_window = QMainWindow()
    main_window.setWindowTitle("旅游景点筛选")

    central_widget = QWidget(main_window)
    main_layout = QHBoxLayout(central_widget)

    left_widget = QWidget(central_widget)
    left_layout = QVBoxLayout(left_widget)
    left_widget.setFixedWidth(400)  # 调整左侧部件的宽度

    right_widget = QWidget(central_widget)
    right_layout = QVBoxLayout(right_widget)

    option_widgets = {}

    options = {
        "游玩时间：": ["请选择", "0.5", "1", "2", "3", "4", "5"],
        "偏好交通方式：": ["请选择", "步行", "单车", "巴士", "火车", "均可"],
        "景点预算：": ["请选择", "0", "<50", "50-100", "100-200", "200-500", "500+"],
        "游玩最关注什么：": ["请选择", "性价比", "避人流", "网红打卡", "游玩体验"],
        "感兴趣类型：": ["山水景观", "商业街", "博物馆", "名胜古迹", "公园游乐园", "休闲度假"],
        "起始点：": read_start_locations(),
        "结束点：": read_end_locations()
    }

    for label_text, options_list in options.items():
        label = QLabel(label_text)
        left_layout.addWidget(label)

        if label_text == "感兴趣类型：":
            list_widget = QListWidget()
            list_widget.setMinimumHeight(200)
            list_widget.setSelectionMode(QListWidget.MultiSelection)
            for option in options_list:
                list_widget.addItem(option)
            left_layout.addWidget(list_widget)
            option_widgets[label_text] = list_widget

        else:
            combo_box = QComboBox()
            for option in options_list:
                combo_box.addItem(option)
            left_layout.addWidget(combo_box)
            option_widgets[label_text] = combo_box
    confirm_button = QPushButton("确定")
    confirm_button.clicked.connect(lambda: confirm_options(option_widgets, right_layout, right_widget))
    left_layout.addWidget(confirm_button)

    main_layout.addWidget(left_widget)

    initial_map_view = QWebEngineView()
    initial_map_view.load(QUrl.fromLocalFile("C:/Users/33549/my_tasks_all/database_final_work/cluster_map.html"))
    initial_map_view.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
    right_layout.addWidget(initial_map_view)

    right_widget.setLayout(right_layout)
    main_layout.addWidget(right_widget)

    central_widget.setLayout(main_layout)
    main_window.setCentralWidget(central_widget)

    main_window.resize(2100, 1400)
    main_window.show()

    sys.exit(app.exec_())


create_gui()



yes


C:\Users\33549\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\33549\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[{'location': '青浦茶物语', 'loctype': '休闲度假', 'counts': 510, 'lon': 121.11671, 'lat': 31.1446, 'comments': 832, 'attitudes': 122, 'reposts': 356, 'price': nan, 'introduction': '删', 'open': nan, 'close': nan, 'animals': nan, 'flowers': nan, 'culture': nan, 'food': nan, 'kids': nan, 'cluster': 4}, {'location': '西山风景区', 'loctype': '山水景观', 'counts': 683, 'lon': 120.924896, 'lat': 31.090778, 'comments': 1264, 'attitudes': 66, 'reposts': 176, 'price': 0.0, 'introduction': '西山又称洞庭西山、金庭镇，是太湖中的第一大岛，岛上既能看烟波飘渺的太湖湖景，也有位列太湖七十二峰之首的缥缈峰，有古村，也有溶洞，可游览的景点很多而且各具特色，农家乐和湖鲜也很吸引人。', 'open': 8.5, 'close': 17.0, 'animals': nan, 'flowers': nan, 'culture': nan, 'food': 1.0, 'kids': nan, 'cluster': 4}, {'location': '金鸡湖', 'loctype': '山水景观', 'counts': 10162, 'lon': 120.773658, 'lat': 31.140913, 'comments': 18868, 'attitudes': 2421, 'reposts': 2412, 'price': 0.0, 'introduction': '金鸡湖位于苏州工业园区，相比古色古香的古城区，这里的高楼林立与现代化展现了苏州的另一面。金鸡湖面积比杭州西湖稍大，每天都有很多游客前来湖畔休闲散步或是购物、吃美食，新人们也喜欢来此拍摄结婚照。', 'open': 0.0, 'close': 24.0, 'animals': nan,

C:\Users\33549\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\33549\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\33549\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\33549\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\33549\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\33549\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KMeans was fitted with